In [ ]:
import ee
import geemap

import math
import pickle
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt

from array import array
from scipy.ndimage import gaussian_filter
from datetime import datetime, timedelta, date
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from pylib.ee.model.harmonic import HarmonicFit
from pylib.ee.collector.sentinel2.l1 import Level1

In [ ]:
# nodata value
nodata = -9999

# repo name
repo = 'methane-detection'

In [ ]:
# get data path
def get_root_path():    
    return os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]

In [ ]:
# get test scenario based on name
def get_scenario( config, name ):
    for plume in config.plumes:
        if plume.name == name:
            return plume
    return None

In [ ]:
# create aoi
def get_aoi( plume ):    
    
    # convert plume location into point 
    location = ee.Geometry.Point( ( plume.longitude, plume.latitude ) )
    buffer = 4900 / 10 * plume.area 
    
    # conbvert buffer to polygon
    region = location.buffer( buffer ).bounds().getInfo()['coordinates'][0]
    return ee.Geometry.Polygon([[region[0],region[1],region[2],region[3]]], None, False)

In [ ]:
# get adjusted date string
def get_date_offset( dt, offset=0 ):    
    dt = dt + timedelta( days=offset )    
    return dt.strftime('%Y-%m-%d') 

In [ ]:
# convert numpy arrays to nodata masked arrays
def get_masked_band_data( data ):
    for key in data.keys():
        data[ key ] = ma.masked_array( data[ key ], mask=data[ key ]==nodata )        
    return data

In [ ]:
def get_band_data( data ):
    # extract channel data from earth engine sample
    return { 'swir1' : np.array( data.get('B11').getInfo() ),
             'swir2' : np.array( data.get('B12').getInfo() ),
             'red' : np.array( data.get('B4').getInfo() ),
             'green' : np.array( data.get('B3').getInfo() ),
             'blue' : np.array( data.get('B2').getInfo() )
    }

In [ ]:
def get_collection( aoi, start_date, end_date ):

    # get collection of cloudmasked s2 l1c images
    collector = Level1()
    return collector.get_data( aoi,
                                start_date,
                                end_date,
                                cloud_mask='s2cloudless',
                                cld_prj_dist=10 ).sort('system:time_start')

In [ ]:
def get_best_scene( aoi, start_date, end_date, **kwargs ):
        
    valid = False
    scene = None
    
    # parse optional args
    target_red = kwargs.get( 'target_red' )
    min_mask_percent = kwargs.get( 'min_mask_percent', 10 )
    min_red_r2 = kwargs.get( 'min_red_r2', 0.95 )

    # get collection
    print ( f'Searching: {start_date} -> {end_date}' )
    collection = get_collection( aoi, start_date, end_date )
    
    # check empty collection
    size = collection.size().getInfo()
    if size > 0:
    
        # convert collection to list
        images = collection.toList( collection.size().getInfo() )

        print ( f'Found {size} scenes ...' )
        for n in range( size ):

            # get image + info
            image = ee.Image( images.get(n) ).float()
            info = image.getInfo()['properties']

            # print acquisition date
            acq_datetime = datetime.utcfromtimestamp( info['system:time_start']/1000 )         
            print ( f'... scene {n+1} of {size}: ' + info['system:index'] ) 

            # extract s2 data within aoi
            data = image.sampleRectangle(region=aoi, defaultValue=nodata)    
            try:
                # convert red channel data to masked array
                red = np.array( data.get('B4').getInfo() ) 
                red = ma.masked_array( red, mask=red==nodata )            
            except BaseException as err: 
                print( err )
                continue

            # get percentage of cloud-masked pixels
            cp = ( np.sum( red.mask ) / ( red.shape[ 0 ] * red.shape[ 1 ] ) ) * 100
            print ( f'...... masked pixels = {cp}%' )
            if cp < min_mask_percent:

                if target_red is not None:

                    r2 = np.ma.corrcoef( red.flatten(), target_red.flatten() )[0,1]
                    print ( f'...... red r2 = {r2}' )
                    if ( r2 > min_red_r2 ) & ( r2 < 1.0 ):
                        valid = True
                        break   

                else:
                    valid = True
                    break

        if valid is True:
            print ( f'...... OK!' )
            scene = {  'data' : get_masked_band_data ( get_band_data( data ) ), 
                       'info' : info,
                       'id' : acq_datetime.strftime( '%Y-%m-%d' )
            }
                                
    return scene

In [ ]:
def get_cross_correlation( event, reference ):
    # get correlation between event and reference channel data
    return { 'swir1' : np.ma.corrcoef( reference[ 'data' ][ 'swir1' ].flatten(), 
                                       event[ 'data' ][ 'swir1' ].flatten()
                        )[0,1],
             'swir2' : np.ma.corrcoef( reference[ 'data' ][ 'swir2' ].flatten(), 
                                       event[ 'data' ][ 'swir2' ].flatten()
                        )[0,1],
             'red' : np.ma.corrcoef( reference[ 'data' ][ 'red' ].flatten(), 
                                     event[ 'data' ][ 'red' ].flatten()
                        )[0,1]
    }

Sentinel-2 SWIR bands are _aliased_ - aliasing often creates large artifacts when computing ratios between these two bands. To avoid this problem, With reference to [Ehret et al. (2022)](https://arxiv.org/abs/2110.11832), a Gaussian filter with sigma parameter = 0.7 is applied to reflectance data prior to any other processing.

In [ ]:
def get_mbsp( swir1, swir2, sigma=0.7 ):
    
    # apply gaussian filter to reduce aliasing artefacts
    swir1 = gaussian_filter( swir1, sigma=sigma )
    swir2 = gaussian_filter( swir2, sigma=sigma )
            
    def linear_regression( x, y ):
    
        """
        Function for least square linear regression with slope output and 
        y-intercept set to 0

        return c,y_int: slope and y-intercept for the linear fit
        """

        xdata=np.zeros([1000000])
        ydata=np.zeros([1000000])
        for i in range( y.shape[0]-1):
            xdata=np.concatenate((xdata,x[i,:]))
            ydata=np.concatenate((ydata,y[i,:]))

        return np.ma.polyfit( xdata, ydata, 1 )

    # compute adjusted B12 reflectance
    c,y_int = linear_regression( swir2, swir1 )
    print("slope=",c)
    print("y-intercept=",y_int)

    return np.divide( ( c * swir2 - swir1 ), swir1 )

In [ ]:
def get_mbmp( event, reference, sigma=0.7 ):

    # get mbsp for event and reference 
    delR1 = get_mbsp( event[ 'data' ][ 'swir1'], event[ 'data' ][ 'swir2'] )
    delR2 = get_mbsp( reference[ 'data' ][ 'swir1'], reference[ 'data' ][ 'swir2'] )
    
    return delR1 - delR2

In [ ]:
def convert_delR_to_omega( delr, sza, vza ):
    
    # get absorption coeffs lut file
    pathname = os.path.join( get_root_path(), 's2/data/S2_full_mdata_poly_10_delr_to_omega.pkl' )
    
    # convert fractional reflectance to methane enhancement in mole/m^2    
    mdata = pickle.load(open( pathname, 'rb'))
    tamf = ( 1.0 / math.cos( math.radians( sza ) ) + 1.0 / math.cos( math.radians (vza) ) )
    
    # lookup closest tamf
    avamf = np.array( list(mdata[ 'S2' ][ 'MBMP' ].keys()) )
    amf = avamf[ np.argmin(abs(avamf - tamf ) ) ]
    #print ("AMF", tamf , amf)
    
    # compute omega
    pol = np.poly1d(mdata[ 'S2' ][ 'MBMP' ][amf] )
    omega= pol(np.log(delr+1))
    return omega

In [ ]:
def plot_output( event, reference, data, title='', cmap='rainbow', gamma = 1.5, vrange=None ):
    
    # create figure
    fig= plt.figure(figsize=(12, 6))
    ax=plt.subplot(1,2,1)
    plt.sca(ax)

    # plot image data
    im = ax.imshow( data, cmap=cmap)    
    if vrange is not None:
        im.set_clim( ( vrange[ 0 ], vrange[ 1 ] ) )
    
    # create an axes on the right side of ax
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.05)
    fig.colorbar(im, cax=cax)

    # get cross correlation
    corr = get_cross_correlation( event, reference )
    
    # plot central location marker
    ax.plot( delR.shape[0]/2 , delR.shape[1]/2, marker = 'x' , color = 'white' )
    
    # set title
    title = title + 'Event: ' + event[ 'id' ] + ' - Reference: ' + reference[ 'id' ]
    ax.set_title( title )

    #plt.gca().axes.get_yaxis().set_visible(False)
    #plt.gca().axes.get_xaxis().set_visible(False)
    
    def plot_sub_image( axx, image, title, cmap=None ):
            
        im = axx.imshow( image, cmap=cmap )    
        mdata = np.where( image.mask == False, image, np.nan)
                
        im.set_clim(( np.nanpercentile(mdata,1), np.nanpercentile(mdata,99)))
        axx.set_title( title )
        axx.axes.get_yaxis().set_visible(False)
        axx.axes.get_xaxis().set_visible(False)
        
        return

    # plot reference swir2
    plot_sub_image( plt.subplot(2,4,8), 
                    reference[ 'data'][ 'swir2'],
                    'Reference B12',
                   cmap='gray'
                    
    )
    
    # plot event swir2
    plot_sub_image( plt.subplot(2,4,7), 
                    event[ 'data'][ 'swir2'],
                    'Event B12 | B11 R2: %6.2f'%corr[ 'swir1' ],
                   cmap='gray'
    )
    
    
    # plot event rgb
    plot_sub_image( plt.subplot(2,4,3), 
                    np.dstack( ( event[ 'data'][ 'red'] * gamma,
                               event[ 'data'][ 'green'] * gamma, 
                               event[ 'data'][ 'blue'] * gamma )
                    ),
                    'Event RGB | Red R2: %6.2f'%corr[ 'red' ]
    )

    
    # plot main day rgb
    plot_sub_image( plt.subplot(2,4,4), 
                    np.dstack( ( reference[ 'data'][ 'red'] * gamma, 
                               reference[ 'data'][ 'green'] * gamma, 
                               reference[ 'data'][ 'blue'] * gamma )
                    ),
                    'Reference RGB'
    )
            
    fig.tight_layout()
    return

In [ ]:
def get_background_images( aoi,
                          start_date,
                          end_date,
                          order=1 ):    
    # channel lookup
    s2_lut = { 'B2' : 'blue', 
               'B3' : 'green', 
               'B4' : 'red', 
               'B11' : 'swir1', 
               'B12' : 'swir2'
    }
    print( f'Assembling time series {start_date} -> {end_date} ...' )
    data = {}
    
    # get s2 collection     
    collection = get_collection( aoi, 
                                start_date,
                                end_date 
    )
    
    # select and rename rgb and swir channels
    collection = collection.select( 
                        list ( s2_lut.keys() ), 
                        list ( s2_lut.values() )
    )
         
    # iterate over bands
    dates = [ end_date ]
    for band in list ( s2_lut.values() ):
        
        # fit harmonic model to time series
        print( f'... generating {band} reference image' )
        coeffs, result = HarmonicFit.get_coefficients( collection, 
                                                       band,
                                                       order=order 
        )
        
        # compute fitted values for event date
        forecast = HarmonicFit.get_model_time_series( dates, 
                                                    coeffs, 
                                                    order=order
        ).first().select( 'fitted' )
    
        # extract fitted values
        forecast = forecast.setDefaultProjection( collection.first().select( band ).projection() )
        image = np.squeeze( geemap.ee_to_numpy( forecast, region=aoi ) )
        
        # mask invalid extrapolated values
        data[ band ] = ma.masked_array( image, mask=( image < 0.0 ) | ( image > 1.0 ) )
        print( '... OK!' )
        
            
    return { 'data' : data,
             'id' : f'{start_date}->{end_date}'
           }